In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import os

os.chdir('C:/Users/MINSEOK/Desktop/대학생활/대외활동/kaggle/house-prices-advanced-regression-techniques')
house_price=pd.read_csv('train.csv')
final_test_data=pd.read_csv('test.csv')
sample_sub=pd.read_csv('sample_submission.csv')

In [2]:
num_col=list(house_price.describe().columns)
num_col.remove('Id')
print(num_col)
print(type(num_col))

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
<class 'list'>


In [3]:
cat_col=list(house_price.columns)
for col in num_col:
  cat_col.remove(col)
print(cat_col)

['Id', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [4]:
tr_col=['GarageArea','GarageCars','WoodDeckSF','OpenPorchSF','MSSubClass','LotFrontage']

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(house_price[tr_col], house_price['SalePrice'], test_size=0.2, shuffle=True)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

(1168, 6) (1168,) (292, 6) (292,)


In [76]:
from sklearn.model_selection import KFold

def Stacking_Data_Loader(model, model_name, X_train, Y_train, X_test, fold):
    '''
    Put your train, test datasets and fold value!
    This function returns train, test datasets for stacking ensemble :)
    '''

    stk = KFold(n_splits = fold, shuffle = True)
    
    # Declaration Pred Datasets
    train_fold_pred = np.zeros((X_train.shape[0], 1))
    test_pred = np.zeros((X_test.shape[0], fold))
    
    for counter, (train_index, valid_index) in enumerate(stk.split(X_train, Y_train)):
        x_train= X_train.iloc[train_index,:]
        y_train= Y_train[train_index]
        x_valid= X_train.iloc[valid_index,:]
        y_valid= Y_train[valid_index]

        print('------------ Fold', counter+1, 'Start! ------------')
        if model_name == 'cat':
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)])
        elif model_name == 'xgb':
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric = 'auc', verbose = 500, early_stopping_rounds = 200)
        else:
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric = 'auc', verbose = 100, early_stopping_rounds = 200)
            
        print('------------ Fold', counter+1, 'Done! ------------')
        
        train_fold_pred[valid_index] = model.predict(x_valid).reshape(-1,1)
        test_pred[:, counter] = model.predict(X_test)
    
    test_pred_mean = np.mean(test_pred, axis = 1)

    print('Done!')
    
    return train_fold_pred, test_pred_mean


In [8]:
import xgboost as xgb

xgb1_params = {
      'tree_method' : 'gpu_hist', 
      'learning_rate' : 0.01,
      'n_estimators' : 50000,
      'colsample_bytree' : 0.3,
      'subsample' : 0.75,
      'reg_alpha' : 19,
      'reg_lambda' : 19,
      'max_depth' : 5, 
      'predictor' : 'gpu_predictor'
}

xgb1=xgb.XGBRegressor(**xgb1_params)

In [77]:
Stacking_Data_Loader(xgb1, 'xgb', X_train=X_train, Y_train=y_train, X_test=X_test, fold=3)


------------ Fold 1 Start! ------------
[0]	validation_0-auc:nan


c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or 

[200]	validation_0-auc:nan
------------ Fold 1 Done! ------------
------------ Fold 2 Start! ------------
[0]	validation_0-auc:nan


c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or 

[199]	validation_0-auc:nan
------------ Fold 2 Done! ------------
------------ Fold 3 Start! ------------
[0]	validation_0-auc:nan


c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or 

[199]	validation_0-auc:nan
------------ Fold 3 Done! ------------
Done!


c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\MINSEOK\anaconda3\envs\torch\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or 

(array([[180672.734375],
        [181032.15625 ],
        [181856.203125],
        ...,
        [181664.15625 ],
        [181066.5625  ],
        [181178.203125]]),
 array([179939.38541667, 180521.66666667, 180093.28645833, 180053.92708333,
        180116.51041667, 179611.97916667, 180053.92708333, 180053.92708333,
        180521.66666667, 179872.51041667, 180521.66666667, 180116.51041667,
        180093.28645833, 179993.734375  , 179934.234375  , 180053.92708333,
        179993.734375  , 179946.79166667, 180381.671875  , 180521.66666667,
        179611.97916667, 180381.671875  , 179611.97916667, 180521.66666667,
        180053.92708333, 180093.28645833, 179994.69791667, 179611.97916667,
        179993.734375  , 180116.51041667, 179611.97916667, 180521.66666667,
        180053.92708333, 179611.97916667, 179838.21354167, 180053.92708333,
        180053.92708333, 179946.79166667, 179934.234375  , 179872.51041667,
        180093.28645833, 180116.51041667, 180053.92708333, 179993.734375  ,

In [ ]:
train_fold_pred, test_pred_mean